<a href="https://colab.research.google.com/github/rhiraga/colab/blob/main/mitsubishi_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# CSVファイルを読み込む（ファイル名を適宜変更してください）
file_path = "final_result.csv"
df = pd.read_csv(file_path)
actual_column_name = 'wav'
df['wav'] = df['wav'].str.extract(r'([A-Z]+_[A-Z]+)')
# print(df.head())
# wav_sorted_df = df.groupby('wav').apply(lambda x: x.sort_values(by='wav')).reset_index(drop=True)
wav_sorted_df = df.groupby('wav', group_keys=False).apply(lambda x: x.sort_values(by='wav')).reset_index(drop=True)
# wav_sorted_df = df.groupby('wav').apply(lambda x: x.sort_values(by='wav')[x.columns]).reset_index(drop=True)

# 結果を表示
# print(wav_sorted_df.head())

wav_sorted_df.to_csv("wav_sorted.csv", index=False)

<ipython-input-10-b7f68866e937>:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  wav_sorted_df = df.groupby('wav', group_keys=False).apply(lambda x: x.sort_values(by='wav')).reset_index(drop=True)


# 新しいセクション

In [ ]:
import pandas as pd

def sort_subjects(input_file, output_file):
    # CSVを読み込む
    df = pd.read_csv(input_file)

    # 'subject' カラムでソートし、'wav' の順序を保持
    df = df.sort_values(by=['wav', 'subject'])

    # 結果をCSVとして保存
    df.to_csv(output_file, index=False)

def count_subject_occurrences(input_file, output_file):
    # CSVを読み込む
    df = pd.read_csv(input_file)

    # 'wav' と 'subject' ごとの出現回数をカウント
    count_df = df.groupby(['wav', 'subject']).size().reset_index(name='count')

    # 結果をCSVとして保存
    count_df.to_csv(output_file, index=False)
# 実行例
# input_file = "precision.csv"  # 入力CSVファイル名
# output_file = "sorted_subjects.csv"  # 出力CSVファイル名
# sort_subjects(input_file, output_file)
input_file = "sorted_subjects.csv"
output_file = "count_subjects.csv"
count_subject_occurrences(input_file, output_file)

In [ ]:
def transform_precision_csv(input_file, output_file):
    # CSVを読み込む
    df = pd.read_csv(input_file)

    # 'subject' ごとに '_1', '_2', '_3', '_4' を付加した新しいカラム名を作成
    df['subject'] = df['subject'].astype(str)  # subjectを文字列化
    df['subject_index'] = df.groupby(['wav', 'subject']).cumcount() + 1
    df['subject'] = df['subject'] + '_' + df['subject_index'].astype(str)

    # ピボット形式で変換: 'subject' を列名、'precision' を値とする
    transformed_df = df.pivot(index='wav', columns='subject', values='precision')

    # インデックスをリセットし、'wav' を列に戻す
    transformed_df.reset_index(inplace=True)

    # カラム名を整える（'wav' を 'timbre' に変更）
    transformed_df.rename(columns={'wav': 'timbre'}, inplace=True)

    # 結果をCSVとして保存
    transformed_df.to_csv(output_file, index=False)

input_file = "precision.csv"
output_file = "transformed_precision.csv"
transform_precision_csv(input_file, output_file)

In [ ]:
import pandas as pd

def transform_metric_csv(input_file, output_file_prefix):
    # CSVを読み込む
    df = pd.read_csv(input_file)

    # 're' カラムを削除
    if 're' in df.columns:
        df = df.drop(columns=['re'])

    # 変換対象のメトリクス
    metrics = ['precision', 'recall', 'f-measure', 'cemgil', 'confidence', 'user_taps']

    # 各メトリクスについて変形と出力を実行
    for metric in metrics:
        # 'subject' ごとに '_1', '_2', '_3', '_4' を付加した新しいカラム名を作成
        temp_df = df[['wav', 'subject', metric]].copy()
        temp_df['subject'] = temp_df['subject'].astype(str)  # subjectを文字列化
        temp_df['subject_index'] = temp_df.groupby(['wav', 'subject']).cumcount() + 1
        temp_df['subject'] = temp_df['subject'] + '_' + temp_df['subject_index'].astype(str)

        # ピボット形式で変換: 'subject' を列名、メトリクスを値とする
        transformed_df = temp_df.pivot(index='wav', columns='subject', values=metric)

        # インデックスをリセットし、'wav' を列に戻す
        transformed_df.reset_index(inplace=True)

        # カラム名を整える（'wav' を 'timbre' に変更）
        transformed_df.rename(columns={'wav': 'timbre'}, inplace=True)

        # 結果をCSVとして保存
        output_file = f"{output_file_prefix}_{metric}.csv"
        transformed_df.to_csv(output_file, index=False)

def sort_subjects(input_file, output_file):
    # CSVを読み込む
    df = pd.read_csv(input_file)

    # 'subject' カラムでソートし、'wav' の順序を保持
    df = df.sort_values(by=['wav', 'subject'])

    # 結果をCSVとして保存
    df.to_csv(output_file, index=False)

def count_subject_occurrences(input_file, output_file):
    # CSVを読み込む
    df = pd.read_csv(input_file)

    # 'wav' と 'subject' ごとの出現回数をカウント
    count_df = df.groupby(['wav', 'subject']).size().reset_index(name='count')

    # 結果をCSVとして保存
    count_df.to_csv(output_file, index=False)

# 実行例
input_file = "final_result.csv"  # 入力CSVファイル名
output_file_prefix = "transformed_result"  # 出力CSVファイルの接頭辞

# 各メトリクスについて変形を実行
transform_metric_csv(input_file, output_file_prefix)


In [ ]:
import pandas as pd

def transform_metric_csv(input_file, output_file_prefix):
    # CSVを読み込む
    df = pd.read_csv(input_file)

    # 're' カラムを削除
    if 're' in df.columns:
        df = df.drop(columns=['re'])

    # 変換対象のメトリクス
    metrics = ['precision', 'recall', 'f-measure', 'cemgil', 'confidence', 'user_taps']

    # 各メトリクスについて変形と出力を実行
    for metric in metrics:
        # 'subject' ごとに '_1', '_2', '_3', '_4' を付加した新しいカラム名を作成
        temp_df = df[['wav', 'subject', metric]].copy()
        temp_df['subject'] = temp_df['subject'].astype(str)  # subjectを文字列化
        temp_df['subject_index'] = temp_df.groupby(['wav', 'subject']).cumcount() + 1
        temp_df['subject'] = temp_df['subject'] + '_' + temp_df['subject_index'].astype(str)

        # ピボット形式で変換: 'subject' を列名、メトリクスを値とする
        transformed_df = temp_df.pivot(index='wav', columns='subject', values=metric)

        # インデックスをリセットし、'wav' を列に戻す
        transformed_df.reset_index(inplace=True)

        # カラム名を整える（'wav' を 'timbre' に変更）
        transformed_df.rename(columns={'wav': 'timbre'}, inplace=True)

        # 欠損値（NaN）を 0 または適切なデフォルト値で埋める
        transformed_df.fillna(0, inplace=True)

        # 結果をCSVとして保存
        output_file = f"{output_file_prefix}_{metric}.csv"
        transformed_df.to_csv(output_file, index=False)

def sort_subjects(input_file, output_file):
    # CSVを読み込む
    df = pd.read_csv(input_file)

    # 'subject' カラムでソートし、'wav' の順序を保持
    df = df.sort_values(by=['wav', 'subject'])

    # 結果をCSVとして保存
    df.to_csv(output_file, index=False)

def count_subject_occurrences(input_file, output_file):
    # CSVを読み込む
    df = pd.read_csv(input_file)

    # 'wav' と 'subject' ごとの出現回数をカウント
    count_df = df.groupby(['wav', 'subject']).size().reset_index(name='count')

    # 結果をCSVとして保存
    count_df.to_csv(output_file, index=False)

# 実行例
input_file = "final_result.csv"  # 入力CSVファイル名
output_file_prefix = "transformed_result"  # 出力CSVファイルの接頭辞

# 各メトリクスについて変形を実行
transform_metric_csv(input_file, output_file_prefix)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

# CSVファイルを読み込む（ファイル名を適宜変更してください）
file_path = "final_result.csv"
df = pd.read_csv(file_path)
actual_column_name = 'wav'
df['wav'] = df['wav'].str.extract(r'([A-Z]+_[A-Z]+)')
#print(df.head())

# 平均値を計算するカラムリスト
columns_to_average = ['precision', 'recall', 'f-measure', 'cemgil', 'confidence', 'user_taps']

# 'wav'ごとに指定カラムの平均値を計算
averaged_df = df.groupby('wav')[columns_to_average].mean().reset_index()

# 結果を表示
print(averaged_df)

# 必要に応じてCSVに保存
averaged_df.to_csv("averaged_results.csv", index=False)


       wav  precision    recall  f-measure    cemgil  confidence  user_taps
0  ACC_ACC   0.741630  0.693638   0.688333  0.690001    2.732143  31.339286
1  ACC_APF   0.769954  0.709821   0.720247  0.716254    2.785714  30.053571
2  APF_APF   0.770529  0.719866   0.721051  0.720008    2.678571  31.267857
3  REC_ACC   0.797561  0.739844   0.748438  0.738608    2.857143  32.357143
4  REC_APF   0.780525  0.712054   0.727718  0.724194    2.875000  29.714286
5  REC_REC   0.735451  0.667969   0.683117  0.690410    2.767857  29.339286
6  SHM_ACC   0.782495  0.722656   0.733063  0.725503    3.089286  30.125000
7  SHM_APF   0.773438  0.703125   0.715712  0.714916    3.071429  29.750000
8  SHM_REC   0.766448  0.688616   0.706630  0.705697    3.053571  28.946429
9  SHM_SHM   0.787927  0.717634   0.729741  0.729382    2.875000  30.214286
